#Carga de datos y division


In [1]:
import pandas as pd
from sklearn.utils import resample

# Cargar datos
url = "https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L07_sentimientos/data/Amazon.csv"
data = pd.read_csv(url)

# Verificar las columnas y una muestra de los datos
required_columns = ['reviews.text_esp', 'reviews.rating']
missing_columns = [col for col in required_columns if col not in data.columns]
if missing_columns:
    raise ValueError(f"Las siguientes columnas necesarias están ausentes en los datos: {missing_columns}")

data = data[['reviews.text_esp', 'reviews.rating']].copy()

# Asignar sentimiento: >= 4.0 es positivo, < 4.0 es negativo
data['sentimiento'] = data['reviews.rating'].apply(lambda x: 'positivo' if x >= 4.0 else 'negativo')

# Resumir los datos originales
original_counts = data['sentimiento'].value_counts()
print("Datos originales:")
print(original_counts)

# Separar datos positivos y negativos
positivos = data[data['sentimiento'] == 'positivo']
negativos = data[data['sentimiento'] == 'negativo']

# Balancear duplicando los datos de la clase minoritaria
if len(positivos) > len(negativos):
    negativos = resample(negativos, replace=True, n_samples=len(positivos), random_state=42)
else:
    positivos = resample(positivos, replace=True, n_samples=len(negativos), random_state=42)

# Concatenar los datos equilibrados
data = pd.concat([positivos, negativos])

# Resumir los datos finales
#final_counts = data['sentimiento'].value_counts()
#added_data = final_counts - original_counts.reindex(final_counts.index, fill_value=0)

# Mostrar resumen
print("\nDatos añadidos:")
print(added_data)

#print("\nDistribución final de los datos:")
#print(final_counts)

# Mostrar una muestra de los datos procesados
print("\nMuestra de datos equilibrados:")
print(data.head())

Datos originales:
sentimiento
negativo    1200
positivo     800
Name: count, dtype: int64

Datos añadidos:
sentimiento
positivo    400
negativo      0
Name: count, dtype: int64

Distribución final de los datos:
sentimiento
positivo    1200
negativo    1200
Name: count, dtype: int64

Muestra de datos equilibrados:
                                      reviews.text_esp  reviews.rating  \
102  La tableta fue excelente, el rendimiento y el ...             5.0   
435  Me gusta recomendar este producto para futuros...             4.0   
270  Atenuación de luces manos libres. Música a la ...             5.0   
106  Compré esta tableta para mis hijos, me encanta...             5.0   
71   ¡Gran dispositivo! Respuesta rápida con contro...             5.0   

    sentimiento  
102    positivo  
435    positivo  
270    positivo  
106    positivo  
71     positivo  


# Preprocesamiento y generacion de embeddings


In [7]:
!pip install num2words

!pip install spacy
!python -m spacy download es_core_news_sm

import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from multiprocessing import Pool
import math
import spacy
from num2words import num2words


# Cargar modelo de Spacy para el preprocesamiento
nlp = spacy.load('es_core_news_sm')

# Definir stop words adicionales
stop_words_adicionales = {"\u00a1", "-", "\u2014", "http", "<", ">"}
for palabra in stop_words_adicionales:
    nlp.Defaults.stop_words.add(palabra)

# Función para procesar texto
def procesar_texto(texto):
    oraciones = texto.split('\n')
    total_oraciones = len(oraciones)
    oraciones_tokenizadas = []

    for oracion in oraciones:
        tokens = []
        doc = nlp(oracion)
        for token in doc:
            if token.is_digit:
                try:
                    numero = int(token.text)
                    palabra_letras = num2words(numero, lang='es')
                    tokens.append(palabra_letras)
                except ValueError:
                    pass
            else:
                lemma = token.text.lower()
                if lemma and lemma not in stop_words_adicionales:
                    tokens.append(lemma)
        oraciones_tokenizadas.append(" ".join(tokens))

    return total_oraciones, oraciones_tokenizadas

# Paralelizar el preprocesamiento del texto
def procesar_textos_en_paralelo(data_slice):
    resultados = [procesar_texto(texto) for texto in data_slice['reviews.text_esp']]
    total_oraciones, oraciones_tokenizadas = zip(*resultados)
    return list(total_oraciones), list(oraciones_tokenizadas)

# Inicializar el modelo BERT
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", ignore_mismatched_sizes=True)

# Función para obtener el embedding de una oración
def obtener_embedding_oracion(oracion):
    inputs = tokenizer(oracion, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Función para obtener el embedding promedio de una reseña completa
def obtener_embedding_resena(oraciones_tokenizadas):
    embeddings_oraciones = [obtener_embedding_oracion(oracion) for oracion in oraciones_tokenizadas]
    return np.mean(embeddings_oraciones, axis=0) if embeddings_oraciones else np.zeros(model.config.hidden_size)

# Paralelizar la generación de embeddings
def procesar_rango(data_slice):
    embeddings = [obtener_embedding_resena(oraciones) for oraciones in data_slice['oraciones_tokenizadas']]
    return embeddings

# Función para dividir los datos en subconjuntos para procesamiento paralelo
def dividir_datos(data, n_cores):
    longitud = len(data)
    p_mas = len(data) % n_cores
    numero_datos = math.floor(longitud / n_cores)
    limites = [(i * numero_datos + min(i, p_mas), (i + 1) * numero_datos + min(i + 1, p_mas)) for i in range(n_cores)]
    return limites

if __name__ == '__main__':
    n_cores = 7


    # Dividir datos para preprocesamiento
    limites = dividir_datos(data, n_cores)
    data_slices = [data.iloc[lim_inf:lim_sup] for lim_inf, lim_sup in limites]

    # Preprocesar textos en paralelo
    with Pool(n_cores) as pool:
        resultados_preprocesamiento = pool.map(procesar_textos_en_paralelo, data_slices)

    # Consolidar resultados del preprocesamiento
    total_oraciones = []
    oraciones_tokenizadas = []
    for total, tokenizadas in resultados_preprocesamiento:
        total_oraciones.extend(total)
        oraciones_tokenizadas.extend(tokenizadas)

    data['total_oraciones'] = total_oraciones
    data['oraciones_tokenizadas'] = oraciones_tokenizadas

    # Dividir datos para generación de embeddings
    limites = dividir_datos(data, n_cores)
    data_slices = [data.iloc[lim_inf:lim_sup] for lim_inf, lim_sup in limites]

    # Generar embeddings en paralelo
    with Pool(n_cores) as pool:
        resultados_embeddings = pool.map(procesar_rango, data_slices)

    # Consolidar resultados de embeddings
    embeddings = [embedding for resultado in resultados_embeddings for embedding in resultado]
    data['embedding_resena'] = embeddings

    # Convertir embeddings a DataFrame y guardar con etiquetas
    embeddings_df = pd.DataFrame(data['embedding_resena'].to_list())
    embeddings_df['sentimiento'] = data['sentimiento'].values
    embeddings_df.to_csv('embeddings_con_etiqueta.csv', index=False)

    print("Embeddings por reseña con etiquetas guardados en 'embeddings_con_etiqueta.csv'.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 71.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embeddings por reseña con etiquetas guardados en 'embeddings_con_etiqueta.csv'.


#Grid search, para definir mejores hiperparametros en el posterior paso


In [2]:
# 01/ 01/ 2025
# este programa realiza una busqued a de hiperparametros para seleccionar la que
# la que arroje mejor precision esto se hace en paralelo combinando diferentes hiperparametros
# este paso con el fin de elegir los mejores hiperparametros para la clasifciacion posterior a eswte paso
#!pip install multiprocess

import itertools
import multiprocess
import time
import numpy as np
from sklearn.metrics import accuracy_score, recall_score
import pandas as pd
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

n_cores = 4

def Nivelacion_de_Cargas(n_cores, lista_inicial):
    lista_final = []
    longitud_li = len(lista_inicial)
    carga = longitud_li // n_cores
    salidas = longitud_li % n_cores
    contador = 0

    for i in range(n_cores):
        if i < salidas:
            carga2 = contador + carga + 1
        else:
            carga2 = contador + carga
        lista_final.append(lista_inicial[contador:carga2])
        contador = carga2
    return lista_final

# Definir parámetros para SVM
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 0.001, 0.01]
}

keys_svm, values_svm = zip(*param_grid_svm.items())
combinations_svm = [dict(zip(keys_svm, v)) for v in itertools.product(*values_svm)]

# Definir parámetros para MLPClassifier (Red Neuronal)
param_grid_rn = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd']
}

keys_rn, values_rn = zip(*param_grid_rn.items())
combinations_rn = [dict(zip(keys_rn, v)) for v in itertools.product(*values_rn)]

# Evaluar conjunto de hiperparámetros para SVM
def evaluate_svm(hyperparameter_set, mejor_result, lock):
    df = pd.read_csv('embeddings_con_etiqueta.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

    for s in hyperparameter_set:
        clf = SVC()
        clf.set_params(C=s['C'], kernel=s['kernel'], gamma=s['gamma'])
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        proce_accuracy = accuracy_score(y_test, y_pred)
        proce_recall = recall_score(y_test, y_pred, pos_label="positivo")

        lock.acquire()
        if proce_accuracy > mejor_result['accuracy']:
            mejor_result['accuracy'] = proce_accuracy
            mejor_result['recall'] = proce_recall
            mejor_result['params'] = s
        lock.release()

# Evaluar conjunto de hiperparámetros para MLPClassifier
def evaluate_rn(hyperparameter_set, mejor_result, lock):
    df = pd.read_csv('embeddings_con_etiqueta.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

    for s in hyperparameter_set:
        clf = MLPClassifier()
        clf.set_params(hidden_layer_sizes=s['hidden_layer_sizes'], activation=s['activation'], solver=s['solver'])
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        proce_accuracy = accuracy_score(y_test, y_pred)
        proce_recall = recall_score(y_test, y_pred, pos_label="positivo")

        lock.acquire()
        if proce_accuracy > mejor_result['accuracy']:
            mejor_result['accuracy'] = proce_accuracy
            mejor_result['recall'] = proce_recall
            mejor_result['params'] = s
        lock.release()


def evaluate_nb(mejor_result, lock):
    df = pd.read_csv('embeddings_con_etiqueta.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

    clf = GaussianNB()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    proce_accuracy = accuracy_score(y_test, y_pred)
    proce_recall = recall_score(y_test, y_pred, pos_label="positivo")

    lock.acquire()
    if proce_accuracy > mejor_result['accuracy']:
        mejor_result['accuracy'] = proce_accuracy
        mejor_result['recall'] = proce_recall
        mejor_result['params'] = None
    lock.release()

if __name__ == '__main__':
    threads = []
    lock = multiprocess.Lock()

    with multiprocess.Manager() as manager:
        mejor_result_svm = manager.dict({'accuracy': 0, 'recall': 0, 'params': None})
        mejor_result_rn = manager.dict({'accuracy': 0, 'recall': 0, 'params': None})
        mejor_result_nb = manager.dict({'accuracy': 0, 'recall': 0, 'params': None})

        start_time = time.perf_counter()

        splits_svm = Nivelacion_de_Cargas(n_cores, combinations_svm)
        splits_rn = Nivelacion_de_Cargas(n_cores, combinations_rn)

        # Crear y ejecutar los procesos
        for i in range(n_cores):
            threads.append(multiprocess.Process(target=evaluate_svm, args=(splits_svm[i], mejor_result_svm, lock)))
            threads.append(multiprocess.Process(target=evaluate_rn, args=(splits_rn[i], mejor_result_rn, lock)))

        # Proceso para GaussianNB (sin splits)
        threads.append(multiprocess.Process(target=evaluate_nb, args=(mejor_result_nb, lock)))

        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()

        finish_time = time.perf_counter()

        print(f"\nMejor accuracy SVM: {mejor_result_svm['accuracy']}, recall: {mejor_result_svm['recall']}, parámetros: {mejor_result_svm['params']}")
        print(f"Mejor accuracy RN: {mejor_result_rn['accuracy']}, recall: {mejor_result_rn['recall']}, parámetros: {mejor_result_rn['params']}")
        print(f"Mejor accuracy NB: {mejor_result_nb['accuracy']}, recall: {mejor_result_nb['recall']}")

        print(f"\nProgram finished in {finish_time - start_time:.2f} seconds")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro


Mejor accuracy SVM: 0.9, recall: 0.9083333333333333, parámetros: {'C': 10, 'kernel': 'rbf', 'gamma': 0.01}
Mejor accuracy RN: 0.9104166666666667, recall: 0.9416666666666667, parámetros: {'hidden_layer_sizes': (100,), 'activation': 'relu', 'solver': 'adam'}
Mejor accuracy NB: 0.7729166666666667, recall: 0.6875

Program finished in 225.60 seconds


#Clasisifcicacion simultanea de modelos, utilizando el primer dataset etiquetado obtenido de la web

In [12]:
import numpy as np
from multiprocessing import Pool
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import time


# utiulizar los mejores hiperparametros dados por grid search previamnet
def entrenar_modelo(tipo_modelo, x_train, y_train, x_test, y_test):
    if tipo_modelo == 'SVM':
        modelo = SVC(C=10, kernel='rbf', gamma =0.01)
    elif tipo_modelo == 'NB':
        modelo = GaussianNB()
    elif tipo_modelo == 'RN':
        modelo = MLPClassifier(hidden_layer_sizes=(50,),activation='relu', solver='adam',max_iter=300)
    else:
        raise ValueError("Tipo de modelo desconocido")

    modelo.fit(x_train, y_train)
    y_pred = modelo.predict(x_test)


    accuracy = accuracy_score(y_test, y_pred)
    return tipo_modelo, accuracy


if __name__ == '__main__':

    archivo_csv = 'embeddings_con_etiqueta.csv'
    datos = pd.read_csv(archivo_csv)

    # Mapeo de etiquetas a valores binarios
    datos['sentimiento_binario'] = datos['sentimiento'].map({'negativo': 0, 'positivo': 1})


    X = datos.iloc[:, :-1].select_dtypes(include=['float', 'int']).values  # Convertir a matriz NumPy
    y = datos['sentimiento_binario'].values


    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Asegurar que los datos no sean solo de lectura
    x_train, x_test = np.copy(x_train), np.copy(x_test)
    y_train, y_test = np.copy(y_train), np.copy(y_test)


    modelos = ['SVM', 'NB', 'RN']


    inicio = time.time()

    # Entrenamiento en paralelo
    with Pool(processes=len(modelos)) as pool:
        resultados = pool.starmap(entrenar_modelo, [(modelo, x_train, y_train, x_test, y_test) for modelo in modelos])

    # Mostramos resultados
    for modelo, accuracy in resultados:
        print(f"Modelo: {modelo}, Precisión: {accuracy:.2f}")

    print(f"Tiempo total: {time.time() - inicio:.2f} segundos")



Modelo: SVM, Precisión: 0.90
Modelo: NB, Precisión: 0.77
Modelo: RN, Precisión: 0.91
Tiempo total: 16.74 segundos
